# How the data is generated on Hydra

This data is generated by running a parameter sweep, generating a job for each possible combination of candidate parameter values.

These can be altered by changing the values in the Nix expression:

```
  # Parameter combinations to try
  params_list = attr_sweep {
    place_delay_model = [ "delta" "delta_override" ];
    initial_pres_fac = [ 0.5 2 2.828 4 ];
    astar_fac = [ 1.2 1.5 1.8 2 ];
    first_iter_pres_fac = [ 0 0.5 ];
    pres_fac_mult = [ 1.1 1.2 1.3 ];
    acc_fac = [ 0.5 0.7 1 ];
  };

  # Projects to run with each combination of parameters in params_list
  projects = [ "baselitex" "ibex" "bram-n3" ];
```

See [param-sweep.nix](https://github.com/SymbiFlow/nix-symbiflow/blob/dev/jobs/param-sweep.nix) for the specific parameters.

To run a different sweep:

1. Fork the [repo](https://github.com/SymbiFlow/nix-symbiflow) on GitHub
2. `git clone git@github.com:<YOU>/nix-symbiflow.git`
3. Edit `nix-symbiflow/jobs/param-sweep.nix`
4. Clone [this jobset](https://hydra.vtr.tools/jobset/dusty/param-sweep) and edit to point at your fork. You will need a Hydra account; ask me for one at deweese@google.com.
   - Actions > Clone this jobset
   - Set the value of `nix-symbiflow` at the bottom to `https://github.com/<YOU>/nix-symbiflow.git <your branch>`
   - Apply changes
   - Actions > Evaluate this jobset
5. Copy this colab.
6. Once that completes, run "Runtime > Run all" in this colab.

In [1]:
#@title Run to collect fpga-tool-perf data from each test on Hydra
#@markdown This retrieves data from the latest evaluation from the `param-sweep` jobset on hydra.vtr.tools

import pandas as pd
import requests
import io

import asyncio
import concurrent.futures

# https://stackoverflow.com/a/56434301
import nest_asyncio
nest_asyncio.apply()

def collect_data_for_build(build):
    url = 'https://hydra.vtr.tools/build/{}/download/meta.json'.format(build)
    resp = requests.get(url, headers = { 'Content-Type': 'application/json' })
    resp2 = requests.get('https://hydra.vtr.tools/build/{}/download/options.json'.format(build), headers = { 'Content-Type': 'application/json' })
    if resp.status_code == 200 and resp2.status_code == 200:
      d = resp.json()
      d.update(resp2.json())
      d['url'] = url
      d['build'] = build
      return [d]
    else:
      return []

# uses Hydra's API: https://hydra.nixos.org/build/121799892/download/1/hydra/#chap-api
#@markdown Enter your Hydra username
project = 'dusty' #@param {type:"string"}
#@markdown Enter the identifier of the project you cloned from `param-sweep`
jobset = 'param-sweep' #@param {type:"string"}
#@markdown Now run all cells: Runtime > Run all
resp = requests.get(
    'https://hydra.vtr.tools/jobset/{}/{}/evals'.format(project, jobset),
    headers = { 'Content-Type': 'application/json' })
evals_json = resp.json()
builds = evals_json['evals'][0]['builds'] # get builds from the first (latest) evaluation

async def get_data():
  with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
      loop = asyncio.get_event_loop()
      futures = [
          loop.run_in_executor(
              executor, 
              collect_data_for_build,
              build
          )
          for build in builds
      ]
      data = []
      for d in await asyncio.gather(*futures):
        data += d
      return data
  
loop = asyncio.get_event_loop()
data = loop.run_until_complete(get_data())

# How to select a good set of parameters

Start with parameters that result in the shortest runtime for each project, and keep the parameters that agree.

Then, for the remaining parameters, try values and check that they rank highly among all projects in the table below when sorted by time.

Use the filter to narrow the table if needed.

# Data

In [2]:
#@title Tables for each project

%reload_ext google.colab.data_table
from IPython.core.display import display, HTML

df = pd.json_normalize(data) #.dropna(axis=1, how='all')
df['parameters'] = df['parameters'].str.split('--')
df['time'] = df['runtime.total'] - df['runtime.synthesis']
bram = df[df.project == 'bram-n3']

# Parameters to choose
params = [
  'extra_vpr_flags.acc_fac',
  'extra_vpr_flags.astar_fac',
  'extra_vpr_flags.first_iter_pres_fac',
  'extra_vpr_flags.initial_pres_fac',
  'extra_vpr_flags.place_delay_model',
  'extra_vpr_flags.pres_fac_mult'
]

# Other parameters of interest
common = params + [
'runtime.placement',
'runtime.routing',
'wirelength',
'time',
'clk',
'build'
]

# Choose the main clock for each project
df['clk'] = 0
df.loc[df.project == 'ibex', ['clk']] = df['max_freq.clk.actual']
df.loc[df.project == 'baselitex', ['clk']] = df['max_freq.sys_clk.actual']
df.loc[df.project == 'bram-n3', ['clk']] = df['max_freq.clk.actual']

# Baselines for comparison
baseline = df[(df['extra_vpr_flags.acc_fac'] == 1) &
              (df['extra_vpr_flags.astar_fac'] == 1.2) &
              (df['extra_vpr_flags.first_iter_pres_fac'] == 0) &
              (df['extra_vpr_flags.initial_pres_fac'] == 2.828) &
              (df['extra_vpr_flags.place_delay_model'] == 'delta_override') &
              (df['extra_vpr_flags.pres_fac_mult'] == 1.2)]

baseline_ibex = baseline[baseline.project == 'ibex'].iloc[0]
print("baseline for ibex: time {}, clk {}".format(baseline_ibex.time, baseline_ibex.clk))
baseline_baselitex = baseline[baseline.project == 'baselitex'].iloc[0]
print("baseline for baselitex: time {}, clk {}".format(baseline_baselitex.time, baseline_baselitex.clk))

# Runtime improvement
df['improvement'] = 0
df.loc[df.project == 'ibex', ['improvement']] = 1 - df[df.project == 'ibex'].time / baseline_ibex.time
df.loc[df.project == 'baselitex', ['improvement']] = 1 - df[df.project == 'baselitex'].time / baseline_baselitex.time

# CPD improvement
df['cpd_improvement'] = 0
df.loc[df.project == 'ibex', ['cpd_improvement']] = 1 - 1 / (df[df.project == 'ibex'].clk / baseline_ibex.clk)
df.loc[df.project == 'baselitex', ['cpd_improvement']] = 1 - 1 / (df[df.project == 'baselitex'].clk / baseline_baselitex.clk)

# Views for each project
ibex = df[df.project == 'ibex']
baselitex = df[df.project == 'baselitex']
bram = df[df.project == 'bram-n3']

# Show tables
display(HTML('<h1>ibex</h1>'))
display(ibex[common + ['improvement', 'cpd_improvement']])
display(HTML('<h1>baselitex</h1>'))
display(baselitex[common + ['improvement', 'cpd_improvement']])
display(HTML('<h1>bram</h1>'))
display(bram[common])


baseline for ibex: time 391.911, clk 33.57
baseline for baselitex: time 1168.401, clk 48.276


,extra_vpr_flags.acc_fac,extra_vpr_flags.astar_fac,extra_vpr_flags.first_iter_pres_fac,extra_vpr_flags.initial_pres_fac,extra_vpr_flags.place_delay_model,extra_vpr_flags.pres_fac_mult,runtime.placement,runtime.routing,wirelength,time,clk,build,improvement,cpd_improvement
2,0.7,1.8,0.5,2.828,delta,1.2,28.99,82.67,468661,338.603,30.728,40873,0.136021,-0.092489
4,0.5,2.0,0.5,2.828,delta,1.1,28.76,95.00,464418,356.120,31.292,40908,0.091324,-0.072798
7,0.5,2.0,0.0,0.500,delta,1.3,30.51,125.25,459434,385.436,30.310,43030,0.016522,-0.107555
8,0.5,1.5,0.0,2.000,delta_override,1.3,31.04,96.32,437841,358.103,32.930,43059,0.086264,-0.019435
9,0.5,1.8,0.5,2.000,delta,1.1,29.88,108.46,456570,366.570,30.995,40935,0.064660,-0.083078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1700,1.0,2.0,0.0,2.828,delta,1.3,30.43,85.50,484149,352.362,31.082,43202,0.100913,-0.080046
1703,1.0,1.8,0.0,0.500,delta,1.1,27.94,173.89,439932,433.356,32.485,43005,-0.105751,-0.033400
1704,1.0,1.2,0.5,0.500,delta_override,1.2,31.14,226.01,414300,484.852,33.288,43404,-0.237148,-0.008472
1707,0.5,2.0,0.0,2.828,delta,1.2,31.27,81.99,474298,350.414,30.164,42356,0.105884,-0.112916


,extra_vpr_flags.acc_fac,extra_vpr_flags.astar_fac,extra_vpr_flags.first_iter_pres_fac,extra_vpr_flags.initial_pres_fac,extra_vpr_flags.place_delay_model,extra_vpr_flags.pres_fac_mult,runtime.placement,runtime.routing,wirelength,time,clk,build,improvement,cpd_improvement
3,1.0,2.0,0.5,4.000,delta,1.1,160.64,325.51,1041490,1011.479,47.174,43104,0.134305,-0.023360
6,0.7,1.8,0.5,2.000,delta,1.3,121.57,318.80,1017661,950.384,47.963,40680,0.186594,-0.006526
13,0.5,1.2,0.0,0.500,delta,1.2,181.98,595.24,924117,1266.065,47.880,43036,-0.083588,-0.008271
17,1.0,2.0,0.5,2.828,delta,1.3,189.28,622.25,1059257,1341.023,47.430,42558,-0.147742,-0.017837
21,0.5,1.8,0.5,4.000,delta_override,1.1,203.85,273.66,1042601,991.285,47.486,41617,0.151588,-0.016636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1695,0.5,2.0,0.5,2.000,delta_override,1.2,157.53,272.54,1050155,918.836,47.358,41532,0.213595,-0.019384
1699,0.7,1.5,0.5,2.000,delta,1.3,182.60,354.52,996919,1047.790,48.232,40900,0.103227,-0.000912
1702,0.5,2.0,0.5,0.500,delta_override,1.3,183.23,401.17,1033696,1108.656,47.473,42897,0.051134,-0.016915
1710,0.5,2.0,0.5,0.500,delta,1.1,175.53,540.56,964193,1238.018,47.423,42261,-0.059583,-0.017987


,extra_vpr_flags.acc_fac,extra_vpr_flags.astar_fac,extra_vpr_flags.first_iter_pres_fac,extra_vpr_flags.initial_pres_fac,extra_vpr_flags.place_delay_model,extra_vpr_flags.pres_fac_mult,runtime.placement,runtime.routing,wirelength,time,clk,build
0,1.0,2.0,0.5,4.000,delta,1.3,7.45,15.91,114592,135.446,58.029,43718
1,0.7,2.0,0.5,0.500,delta,1.3,7.46,23.70,113220,140.401,58.014,44009
5,0.7,1.8,0.5,2.000,delta_override,1.3,8.64,33.54,113474,155.493,67.097,43616
14,0.5,1.8,0.0,4.000,delta,1.1,7.63,24.86,115100,148.984,58.121,43610
15,0.7,1.5,0.5,0.500,delta,1.1,8.10,71.12,113735,195.742,59.670,43962
...,...,...,...,...,...,...,...,...,...,...,...,...
1701,0.5,2.0,0.0,2.000,delta_override,1.3,8.36,33.88,113385,156.302,68.271,43707
1705,1.0,1.2,0.5,2.000,delta,1.2,7.67,58.81,117220,180.022,60.063,43833
1706,1.0,1.5,0.5,0.500,delta_override,1.2,8.38,80.10,114555,200.630,67.870,43710
1708,0.7,2.0,0.0,2.828,delta,1.2,6.84,18.42,114801,135.355,57.978,44055


In [3]:
#@title Candidates
#@markdown Sort by `improvement`, and filter out any values that cause other failures. Verify that these parameters work well, and also look at the remaining statistics and plot below.
#@markdown Check `cpd_improvement` and to balance quality losses (negative values.)
both = df[df.project.isin(['ibex', 'baselitex'])]
both.groupby(params)['improvement', 'cpd_improvement'].min().reset_index()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  after removing the cwd from sys.path.


,extra_vpr_flags.acc_fac,extra_vpr_flags.astar_fac,extra_vpr_flags.first_iter_pres_fac,extra_vpr_flags.initial_pres_fac,extra_vpr_flags.place_delay_model,extra_vpr_flags.pres_fac_mult,improvement,cpd_improvement
0,0.5,1.2,0.0,0.5,delta,1.1,-0.450015,-0.027171
1,0.5,1.2,0.0,0.5,delta,1.2,-0.242598,-0.023725
2,0.5,1.2,0.0,0.5,delta,1.3,-0.209438,-0.033464
3,0.5,1.2,0.0,0.5,delta_override,1.1,-0.441514,-0.003304
4,0.5,1.2,0.0,0.5,delta_override,1.2,-0.221818,-0.008903
...,...,...,...,...,...,...,...,...
570,1.0,2.0,0.5,4.0,delta,1.2,0.042681,-0.089475
571,1.0,2.0,0.5,4.0,delta,1.3,-0.055911,-0.094127
572,1.0,2.0,0.5,4.0,delta_override,1.1,0.122788,-0.106351
573,1.0,2.0,0.5,4.0,delta_override,1.2,-0.125902,-0.105003


# Statistics for Runtime and Maximum Frequency

Use the statistics to check that the selected parameters rank well with mean and minimum values, and that quality (maximum clock frequency) doesn't suffer.

The parameters should be chosen by balancing the stability of the improvement across the projects considered and the benefit. For example, if a parameter shows a clear benefit across all projects, it should be used, but if the effect is varies across projects, or the effect is small, it should not be used.

In [4]:
#@title Statistics on each parameter
for p in params:
  display(HTML("<h1>" + p + "</h1>"))
  for x in ['time', 'clk']:
    display(HTML("<h2>" + x + "</h2>"))
    display(HTML("<h3>ibex</h3>"))
    display(ibex.groupby(p)[x].describe())
    display(HTML("<h3>baselitex</h3>"))
    display(baselitex.groupby(p)[x].describe())
    display(HTML("<h3>bram</h3>"))
    display(bram.groupby(p)[x].describe())

,count,mean,std,min,25%,50%,75%,max
extra_vpr_flags.acc_fac,,,,,,,,
0.5,190.0,377.774468,47.727299,307.739,345.04625,363.7915,396.0975,580.190
0.7,189.0,378.278238,47.578479,308.115,347.04600,363.6730,396.5590,565.330
1.0,190.0,380.590221,46.209744,308.017,348.23675,362.7660,401.4295,595.594


,count,mean,std,min,25%,50%,75%,max
extra_vpr_flags.acc_fac,,,,,,,,
0.5,188.0,1115.754505,117.951916,856.214,1033.11075,1101.545,1184.038,1441.937
0.7,189.0,1096.447608,116.411542,831.724,1016.64100,1073.733,1147.801,1581.330
1.0,191.0,1133.620052,135.533777,838.874,1049.08500,1102.290,1187.296,1662.410


,count,mean,std,min,25%,50%,75%,max
extra_vpr_flags.acc_fac,,,,,,,,
0.5,192.0,175.205495,35.778324,137.184,151.31875,164.6185,183.00600,341.914
0.7,192.0,172.577859,33.036306,128.697,151.15925,163.6035,181.11300,320.673
1.0,192.0,172.627193,33.808977,133.200,150.59150,161.6925,180.47275,337.429


,count,mean,std,min,25%,50%,75%,max
extra_vpr_flags.acc_fac,,,,,,,,
0.5,190.0,32.014547,0.880959,29.740,31.32225,32.0590,32.70375,34.317
0.7,189.0,32.003339,0.961672,29.060,31.42900,31.9800,32.62700,34.534
1.0,190.0,31.930205,0.880172,30.231,31.18475,31.9065,32.60975,34.154


,count,mean,std,min,25%,50%,75%,max
extra_vpr_flags.acc_fac,,,,,,,,
0.5,188.0,47.814888,0.459386,45.804,47.50925,47.943,48.1715,48.454
0.7,189.0,47.857556,0.508739,44.248,47.63300,48.003,48.1960,48.530
1.0,191.0,47.854780,0.484012,44.208,47.55550,47.950,48.2215,48.649


...

,count,mean,std,min,25%,50%,75%,max
extra_vpr_flags.pres_fac_mult,,,,,,,,
1.1,190.0,1101.636884,128.098209,875.059,1016.74150,1063.3720,1159.73025,1528.942
1.2,188.0,1086.060410,92.152686,838.874,1028.97800,1080.0665,1139.82025,1348.821
1.3,190.0,1158.007947,136.834368,831.724,1074.24075,1143.1000,1223.17725,1662.410


,count,mean,std,min,25%,50%,75%,max
extra_vpr_flags.pres_fac_mult,,,,,,,,
1.1,192.0,179.447615,39.938153,128.697,154.45000,166.600,189.09925,341.914
1.2,192.0,171.601354,31.939453,129.371,151.00725,161.149,180.19450,298.620
1.3,192.0,169.361578,29.111739,135.446,149.13075,159.994,177.99325,276.522


,count,mean,std,min,25%,50%,75%,max
extra_vpr_flags.pres_fac_mult,,,,,,,,
1.1,188.0,32.250644,0.864646,30.291,31.55775,32.2125,32.91225,34.534
1.2,190.0,31.959747,0.859587,29.740,31.30650,31.9025,32.64575,34.317
1.3,191.0,31.741681,0.927972,29.060,31.04750,31.7950,32.34500,34.224


,count,mean,std,min,25%,50%,75%,max
extra_vpr_flags.pres_fac_mult,,,,,,,,
1.1,190.0,47.915811,0.386844,46.600,47.57975,48.0585,48.21600,48.649
1.2,188.0,47.844112,0.430902,46.148,47.54350,47.9470,48.19625,48.530
1.3,190.0,47.767595,0.598737,44.208,47.44275,47.9395,48.14925,48.582


,count,mean,std,min,25%,50%,75%,max
extra_vpr_flags.pres_fac_mult,,,,,,,,
1.1,192.0,63.353313,4.637518,57.531,58.73250,63.315,68.01825,68.991
1.2,192.0,63.365172,4.644083,57.387,58.82975,63.498,68.06700,68.901
1.3,192.0,63.356250,4.670642,57.359,59.06450,63.409,68.06250,68.897


# Generality

Look at the scatter plot to get an idea of the larger population of configurations, to ensure that parameter values are likely to work well on other projects.

There should be several similar parameters nearby.

In [5]:
#@title Runtime vs. Max Frequency Plot
#@markdown This plot is to visually identify anomalies and trends.
#@markdown Hover over a point to see the parameters used for that sample.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import plotly.graph_objects as go

def label_row(row):
  strings = []
  for p in params:
    strings += [p + ': ' + str(row[p])]
  return ', '.join(strings)

def plot(fig, df, x_axis, y_axis):
  labels = df.loc[:, params]
  fig.add_trace(go.Scatter(x=df[x_axis], y=df[y_axis], mode='markers',
                           #marker_color=df['color'],
                           marker_size=7,
                           text=labels.apply(label_row, axis=1)))

def show_plot(df):
  fig = go.Figure()
  plot(fig, df, 'time', 'clk')
  fig.update_layout(
      height=1000,
      xaxis_title='Run time',
      yaxis_title='Maximum Clock Frequency')
  fig.show()

for p in ['ibex', 'baselitex', 'bram']:
  display(HTML('<h1>' + p + '</h1>'))
  exec('show_plot(' + p + ')')

# Gathering more data

If the data hints that there might be a better value for some parameter, add it and run another evaluation. Tests are cached, so only new parameter combinations will be tested.

# Appendix

In [6]:
#@title List Columns
#@markdown For checking what data is available.
for c in df.columns:
  print(c)

board
build
build_type
carry
cmds
date
design
device
family
maximum_memory_use
optstr
package
parameters
pcf
project
sdc
seed
sources
strategy
top
vendor
wirelength
xdc
url
max_freq.clk.actual
max_freq.clk.hold_violation
max_freq.clk.met
max_freq.clk.requested
max_freq.clk.setup_violation
resources.BRAM
resources.CARRY
resources.DFF
resources.GLB
resources.IOB
resources.LUT
resources.PLL
runtime.bitstream
runtime.checkpoint
runtime.fasm
runtime.fasm2bels
runtime.link design
runtime.optimization
runtime.overhead
runtime.packing
runtime.phys opt design
runtime.placement
runtime.prepare
runtime.reports
runtime.routing
runtime.synthesis
runtime.total
toolchain.vpr.pr_tool
toolchain.vpr.synthesis_tool
versions.vpr
versions.yosys
extra_vpr_flags.acc_fac
extra_vpr_flags.astar_fac
extra_vpr_flags.first_iter_pres_fac
extra_vpr_flags.initial_pres_fac
extra_vpr_flags.place_delay_model
extra_vpr_flags.pres_fac_mult
max_freq.clk200_clk.actual
max_freq.clk200_clk.hold_violation
max_freq.clk200_clk.m